In [58]:
from rtmlib import Wholebody, draw_skeleton
import cv2
from tqdm import tqdm
import json

In [59]:
device = 'cpu'  # cpu, cuda, mps
backend = 'onnxruntime'  # opencv, onnxruntime, openvino
video_path = '../Movies/output_video.mp4'  # Path al video (o 0 per webcam)
output_path = 'output_video.avi'  # Salvataggio del video elaborato
openpose_skeleton = True  # True per lo stile OpenPose, False per MMPose
keypoints_data = []

In [63]:
# Inizializza il modello Wholebody
wholebody = Wholebody(to_openpose=openpose_skeleton,
                      mode='balanced',  # 'performance', 'lightweight', 'balanced'
                      backend=backend, device=device)

# Lettura del video
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Errore nell'aprire il video!")
    exit()

total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) 
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'XVID') 
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

with tqdm(total=total_frames, desc="Processing Frames", unit="frame") as pbar:
    keypoints_list = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        keypoints, scores = wholebody(frame)
        
        frame_obj = {
            "frame": int(cap.get(cv2.CAP_PROP_POS_FRAMES)),
        }
        
        for index, (person_kp, person_score) in enumerate(zip(keypoints, scores)):
                person_keypoints = [{"x": float(kp[0]), "y": float(kp[1]), "score": sc} for kp, sc in zip(person_kp, person_score)]
                frame_obj[f"Person{index+1}"] = person_keypoints
        
        keypoints_list.append(frame_obj)
        img_show = draw_skeleton(frame, keypoints, scores, openpose_skeleton=openpose_skeleton)

        out.write(img_show)

        pbar.update(1)
        
with open("keypoints.json", "w") as f:
    json.dump(keypoints_list, f, indent=4)

# Rilascia le risorse
cap.release()
out.release()


load C:\Users\fedep\.cache\rtmlib\hub\checkpoints\yolox_m_8xb8-300e_humanart-c2c7a14a.onnx with onnxruntime backend
load C:\Users\fedep\.cache\rtmlib\hub\checkpoints\rtmw-x_simcc-cocktail13_pt-ucoco_270e-256x192-fbef0d61_20230925.onnx with onnxruntime backend


Processing Frames: 100%|██████████| 154/154 [05:33<00:00,  2.17s/frame]


In [34]:
print(len(keypoints[0]))

134
